In [8]:
import os
import re
import gzip
# name of channel to be scanned
channel_prefix = "https://www.youtube.com/"
channel = "@StronnyCuttles"
#channel = "@ShibiCottonbum"
#channel = "@ImmyBisou"
#channel = "@IceySnowpaws"
#channel = "@MercyModiste"
#channel = "@AzuraDulait"

In [10]:
try:   os.mkdir(channel)
except:   pass
try:   os.mkdir(f"{channel}/downloaded")
except:   pass
try:   os.mkdir(f"{channel}/processed")
except:   pass
try:   os.mkdir(f"{channel}/transcripts")
except:   pass
# file containing keywords
keywords_file = "keywords.txt"

# file containing thematic searches
thematic_file = "thematic.txt"

# scan channel for videos
# live videos
os.system(f"yt-dlp --get-id --skip-download {channel_prefix+channel} > {channel}/videos.txt")
videos = []
with open(f"{channel}/videos.txt", "r") as f:
    for line in f:
        if line.startswith("Error"):
            continue
        videos.append(line.strip())

# look for filenames in {channel}/downloaded for already downloaded videos
# the file name contains the video id in the following format:
# 【Supermarket Simulator】Definitely not drunk on the job 🐐🍼【VAllure】 [i3rrDSjokF4].webm
# where [i3rrDSjokF4] is the video id
downloaded = []
for file in os.listdir(f"{channel}/downloaded"):
    if file.endswith(".webm"):
        m = re.search(r"\[(.*?)\]", file)
        if m:
            downloaded.append(m.group(1))
            
# do the same for transcripts
transcripts = []
for file in os.listdir(f"{channel}/transcripts"):
    if file.endswith(".srt"):
        m = re.search(r"\[([^\[\]]+)\](?!.*\[)", file)
        if m:
            transcripts.append(m.group(1))


# compare list of videos to videos already downloaded
to_download = list(set(videos) - set(downloaded) - set(transcripts))
for download in to_download:
    print(f"{download}")

bxvGYUQqmeA
C5M2nZUlFOI
rAga_P8IPN8
XzICDn_8-Jk
BKzfzKDH4K4
IjI_vbw789k
h7RxzeuTUrk
vpjDr5Uy4u4
Ittk3Ou_R6w
-lFoCKT-HTQ
ARjYuEluPH8
cNZoxeYDBrM
N5XlaAafWhw
g_3b5GH7dUw
Kn5zueFswok
WPzDPmNU4Ks
S_Q9PvOzBrA
kJi0-CkRM04
Tydd2TwQKes
TezvXw9NcmA
LJQxOBO9R5o
KS_or6k8Fgs
vpE9V6Yq5mk
lW72sUi_yGM
jNsADJMHcH0
-PKAkJYvLlk
t-8BGpSdgcs
n9V9pF55XOI
KXrIrwwU9fE
6y0sq0vGBz8
tKQ7Zc0xElM
ySbug7LOJvc
Mh91eB_zf3I
ato1JYpgKjg
18xCiE-xMXI
X3eS7c7bLBs
ItOLT7FqIkU
dI3Qgiy9wLY
doQ-m-JaOWI
Fc5YPISpWo0
zhLGqcIL4yk
A-5r9T4hKwA
-qlXrLLQkzo
eHacBfPGqGk
B4yGQsVzcNE
AiLh5KvhIt8
J2I6Wss9WsQ
5a0q3yr68IU
FbIAzXp0TNM
tUd0tNFVvT4
QR3yoGDDp_E
J7YwGiY6z2g
2t9uBYvO2kM
T0Jz9yri2aw
l6e818-vgg8
aFttYk8rGN4
6x1Pwqmhdhk
0pAi_PZPiT0
zzRsF_sd1_4
cqtBFwac9hY
Lo0bzApA-mI
mtPS60PrlRY
AlJxvHKxB6s
SkegmwvMD4I
4IBZgfeJd7M
5Oum6Ad7lgU
3rIax65ailI
Nd8iJAiB9NQ
KjCx8ZxanXo
VWY5nDNxgJA
8RIWelNdr-E
XP7BlFSaSYs
AWji_uhsJA4
y57y6sDhUZA
OqUn_IWJLbU
W6ljJwjCQj0
J1yWMGR95r4
1WoJxJcgWfw
D_W1cFW-hL0
MfQ5SKqCfhk
GYLEhqMZ2Ro
AiqIHq74hGk
IB39lKeehAQ
G0-5

In [12]:
newly_downloaded = []
for video in to_download:
    print(f"Downloading {video}")
    output = os.system(f"yt-dlp -f bestaudio -P {channel}/downloaded {channel_prefix}watch?v={video}")
    if output!=0:
        print(f"Error downloading {video}")

In [13]:
#collect file names in {channel}/downloaded
downloaded_files = os.listdir(f"{channel}/downloaded")
for file in downloaded_files:
    if file.endswith(".webm") or file.endswith(".m4a"):
        #result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --output_dir --language English --model large-v3 \"{channel}/transcripts/\"")
        result = os.system(f"whisper \"{channel}/downloaded/{file}\" --device cuda --language English --output_dir \"{channel}/transcripts/\"")
        if result==0:
            #move file to {channel}/processed
            try:
                os.rename(f"{channel}/downloaded/{file}", f"{channel}/processed/{file}")
            except:
                print(f"Error moving {file} to {channel}/processed")

In [14]:
#remove .tsv, .txt, .vtt files, (keeping .srt and .json), and compress json files to .json.gz in {channel}/transcripts
transcript_files = os.listdir(f"{channel}/transcripts")
for file in transcript_files:
    if file.endswith(".tsv") or file.endswith(".txt") or file.endswith(".vtt"):
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
    if file.endswith(".json"):
        with open(f"{channel}/transcripts/{file}", "rb") as f:
            with gzip.open(f"{channel}/transcripts/{file}.gz", "wb") as f_out:
                f_out.writelines(f)
        #remove original json file
        try:
            os.remove(f"{channel}/transcripts/{file}")
        except:
            print(f"Error removing {file}")
        